### Filter Sky Brightness Data
Only keep hourly data, from 1-10-2023 to 1-10-2024

In [3]:
import pandas as pd
import os

directory = r"C:\Users\cxoox\Desktop\sky_brightness"
output_dir = r"C:\Users\cxoox\Desktop\filtered_data"

os.makedirs(output_dir, exist_ok=True)
filtered_kings_park = pd.DataFrame()

if not os.path.exists(directory):
    print(f"Directory does not exist: {directory}")
else:
    for file in os.listdir(directory):
        if not file.endswith('.csv'):
            continue
            
        file_path = os.path.join(directory, file)
        print(f"Processing {file_path}")
        
        try:
            df = pd.read_csv(file_path)
            df = df[df['device_code'].isin(['KP'])]  # Filter device
            
            # Parse datetime with multiple format attempts
            for fmt in ['%d/%m/%Y %H:%M:%S', '%m/%d/%Y %H:%M:%S', 
                       '%d/%m/%y %H:%M', '%Y-%m-%d %H:%M:%S']:
                try:
                    df['received_adjusted'] = pd.to_datetime(df['received_adjusted'], format=fmt)
                    break
                except:
                    continue
            
            if not pd.api.types.is_datetime64_any_dtype(df['received_adjusted']):
                print(f"Datetime parsing failed in {file}")
                continue
                
            # Sort by timestamp
            df = df.sort_values('received_adjusted')
            
            # Create hour-level grouping key
            df['hour_group'] = df['received_adjusted'].dt.strftime('%Y-%m-%d %H:00')
            
            # Keep first observation in each hour group
            df = df.drop_duplicates(subset=['hour_group'], keep='first')
            
            filtered_kings_park = pd.concat([filtered_kings_park, df], ignore_index=True)
            
        except Exception as e:
            print(f"Error processing {file}: {str(e)}")

# Final cleanup
filtered_kings_park = filtered_kings_park.drop(columns=['hour_group'])  # Remove helper column
filtered_kings_park = filtered_kings_park.drop_duplicates()  # Safety check

# Only keep "received_adjusted" and "nsb" columns
filtered_kings_park = filtered_kings_park[['received_adjusted', 'nsb']]

# Save results
output_path = os.path.join(output_dir, "filtered_kings_park.csv")
filtered_kings_park.to_csv(output_path, index=False)
print(f"Saved {len(filtered_kings_park)} records to {output_path}")

Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2023_10.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2023_11.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2023_12.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2024_01.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2024_02.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2024_03.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2024_04.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2024_05.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2024_06.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2024_07.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2024_08.csv
Processing C:\Users\cxoox\Desktop\sky_brightness\GaN-MN_2024_09.csv
Saved 8760 records to C:\Users\cxoox\Desktop\filtered_data\filtered_kings_park.csv


### Concatenate 每日最高香港暑熱指數及平均香港暑熱指數

In [8]:
import pandas as pd

weather_data = ["Daily Maximum Hong Kong Heat Index", 
                "Daily Mean Hong Kong Heat Index", 
                "Daily Mean Wet Bulb Temperature",
                'Daily Mean Dew Point',
                "Daily Mean Amount of Cloud",
                "Daily Mean Pressure",
                "Daily Total Evaporation",
                "Daily Total Rainfall",
                "Daily Mean Relative Humidity",
                "Daily Maximum Temperature All Year",
                "Daily Minimum Temperature All Year",
                "Daily Mean Temperature All Year",
                "Daily Global Solar Radiation",
                "Daily Total Bright Sunshine (hours)",
                "Daily Mean Wind Speed"]

weather_data_path = [r"C:\Users\cxoox\Downloads\daily_KP_MEANHKHI_ALL.csv",
                     r"C:\Users\cxoox\Downloads\daily_KP_MAXHKHI_ALL.csv", 
                     r"C:\Users\cxoox\Downloads\daily_KP_WET_ALL.csv",
                     r"C:\Users\cxoox\Downloads\daily_KP_DEW_ALL.csv",
                     r"C:\Users\cxoox\Downloads\daily_HKO_CLD_ALL.csv",
                     r"C:\Users\cxoox\Downloads\daily_HKO_MSLP_ALL.csv",
                     r"C:\Users\cxoox\Downloads\daily_KP_EVAP_ALL.csv",
                     r"C:\Users\cxoox\Downloads\daily_KP_RF_ALL.csv",
                     r"C:\Users\cxoox\Downloads\daily_KP_RH_ALL.csv",
                     r"C:\Users\cxoox\Downloads\CLMMAXT_KP_.csv",
                     r"C:\Users\cxoox\Downloads\CLMMINT_KP_.csv",
                     r"C:\Users\cxoox\Downloads\CLMTEMP_KP_.csv",
                     r"C:\Users\cxoox\Downloads\daily_KP_GSR_ALL.csv",
                     r"C:\Users\cxoox\Downloads\daily_KP_SUN_ALL.csv",
                     r"C:\Users\cxoox\Downloads\daily_KP_WSPD_ALL.csv"]


# 1. Load the combined filtered data
filtered_kings_park = pd.read_csv(r"C:\Users\cxoox\Desktop\filtered_data\filtered_kings_park.csv")

# Convert the 'received_adjusted' column to a date column, handling invalid values
filtered_kings_park['date'] = pd.to_datetime(
    filtered_kings_park['received_adjusted'],
    errors='coerce'  # Convert invalid dates to NaT
).dt.date

# Drop rows with invalid dates (if any)
filtered_kings_park = filtered_kings_park.dropna(subset=['date'])

# 2. Load the daily mean cloud cover data
for file_path in weather_data_path:
    if not os.path.exists(file_path):
        print(f"File does not exist: {file_path}")
        continue

    # Read the CSV file, skipping the first two rows and using only the relevant columns
    df = pd.read_csv(
        file_path,
        skiprows=2,
        header=0,
        usecols=["年/Year", "月/Month", "日/Day", "數值/Value"]
    )

    # Rename columns for easier handling
    df = df.rename(columns={
        "年/Year": "Year",
        "月/Month": "Month",
        "日/Day": "Day",
        "數值/Value": weather_data[weather_data_path.index(file_path)]
    })

    # Convert Year, Month, and Day to integers, handling invalid values
    for col in ["Year", "Month", "Day"]:
        df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')  # Int64 supports NaN

    # Combine Year, Month, and Day into a single date column, handling invalid values
    df['date'] = pd.to_datetime(
        df[["Year", "Month", "Day"]].astype(str).agg("-".join, axis=1),
        format="%Y-%m-%d",
        errors='coerce'  # Convert invalid dates to NaT
    ).dt.date

    # Drop rows with invalid dates (if any)
    df = df.dropna(subset=['date'])
    
    filtered_kings_park = pd.merge(
        filtered_kings_park,
        df[['date', weather_data[weather_data_path.index(file_path)]]],
        on='date',
        how='left'
    )
    

# 5. Save the merged data
output_file_path = r"C:\Users\cxoox\Desktop\filtered_data\merged.csv"
filtered_kings_park.to_csv(output_file_path, index=False)
print(f"Merge completed! Data saved to {output_file_path}")
print("First 5 rows of the merged data:")
print(filtered_kings_park.head())

Merge completed! Data saved to C:\Users\cxoox\Desktop\filtered_data\merged.csv
First 5 rows of the merged data:
     received_adjusted  nsb        date Daily Maximum Hong Kong Heat Index  \
0  2023-10-01 08:00:10  0.0  2023-10-01                               28.0   
1  2023-10-01 09:00:10  0.0  2023-10-01                               28.0   
2  2023-10-01 10:00:10  0.0  2023-10-01                               28.0   
3  2023-10-01 11:00:10  0.0  2023-10-01                               28.0   
4  2023-10-01 12:00:10  0.0  2023-10-01                               28.0   

   Daily Mean Hong Kong Heat Index Daily Mean Wet Bulb Temperature  \
0                             30.0                            26.1   
1                             30.0                            26.1   
2                             30.0                            26.1   
3                             30.0                            26.1   
4                             30.0                            26.1   